# 🍏 Observability & Tracing Demo with `azure-ai-projects` and `azure-ai-inference` 🍎

> 📚 **For developers and learners**: Refer to the official Azure AI Foundry observability documentation: [https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability)

Welcome to this **Health & Fitness**-themed notebook, where we'll explore how to set up **observability** and **tracing** for:

1. **Basic LLM calls** using an `AIProjectClient`.
2. **Multi-step** interactions using an **Agent** (such as a Health Resource Agent).
3. **Tracing** your local usage in **console** (stdout) or via an **OTLP endpoint** (like **Prompty** or **Aspire**).
4. Sending those **traces** to **Azure Monitor** (Application Insights) so you can view them in **Azure AI Foundry**.

> **Disclaimer**: This is a fun demonstration of AI and observability! Any references to workouts, diets, or health routines in the code or prompts are purely for **educational** purposes. Always consult a professional for health advice.

## Contents
1. **Initialization**: Setting up environment, creating clients.
2. **Basic LLM Call**: Quick demonstration of retrieving model completions.
3. **Connections**: Listing project connections.
4. **Observability & Tracing**
   - **Azure Monitor** tracing: hooking up to Application Insights
   - **Verifying** your traces in Azure AI Foundry
5. **Agent-based Example**:
   - Creating a simple "Health Resource Agent" referencing sample docs.
   - Multi-turn conversation with tracing.
   - Cleanup.

<img src="./seq-diagrams/1-observability.png" width="75%"/>

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initialization & Setup
**Prerequisites**:
- A `.env` file containing `AI_FOUNDRY_PROJECT_ENDPOINT` (and optionally `MODEL_DEPLOYMENT_NAME`).
- Roles/permissions in Azure AI Foundry that let you do inference & agent creation.
- A local environment with `azure-ai-projects`, `azure-ai-inference`, `opentelemetry` packages installed.

**What we do**:
- Load environment variables.
- Initialize `AIProjectClient`.
- Check that we can talk to a model (like `gpt-4o`).

In [ ]:
import os
import sys
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, CompletionsFinishReason

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / '.env'  # Adjust path as needed
load_dotenv(env_path)

project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
tenant_id = os.environ.get("TENANT_ID")
if not project_endpoint:
    raise ValueError("🚨 AI_FOUNDRY_PROJECT_ENDPOINT not set in .env.")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize AIProjectClient with simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    print("✅ Successfully created AIProjectClient!")
except Exception as e:
    print(f"❌ Error creating AIProjectClient: {e}")
    print("💡 Please complete the browser authentication prompt that should appear")

## 2. Basic LLM Call
We'll do a **quick** chat completion request to confirm everything is working. We'll ask a simple question: "How many feet are in a mile?"

In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    # Use the correct Azure AI Projects SDK pattern
    print("🔄 Getting OpenAI client from Azure AI Project...")
    print(f"🤖 Using model: {model_deployment_name}")
    
    # Get OpenAI client using the correct method
    openai_client = project_client.get_openai_client(api_version="2024-10-21")
    
    # Create chat completion using OpenAI client pattern
    response = openai_client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            {"role": "system", "content": "You are a helpful health assistant"},
            {"role": "user", "content": "How to be healthy in one sentence?"}
        ]
    )
    
    print("✅ Successfully created chat completion!")
    print(f"🤖 Assistant: {response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ An error occurred: {str(e)}")
    print("💡 Troubleshooting tips:")
    print("  - Ensure your Azure AI Project has OpenAI connections configured")
    print("  - Verify your MODEL_DEPLOYMENT_NAME is correctly deployed")
    print("  - Check that you have proper permissions to access the model")
    print("  - Make sure you're using the latest azure-ai-projects SDK version")

## 3. Observability & Tracing

In [ ]:
# # Install packages exactly as specified in Microsoft documentation if not installed using requirements.txt
!pip install azure-ai-projects azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2

## 3.1 Enable OpenTelemetry for Azure AI Inference
We set environment variables to ensure:
1. **Prompt content** is captured (optional!)
2. The **Azure SDK** uses OpenTelemetry as its tracing implementation.
3. We call `AIInferenceInstrumentor().instrument()` and `OpenAIInstrumentor().instrument()` to patch and enable the instrumentation.


In [ ]:
import os
from azure.ai.inference.tracing import AIInferenceInstrumentor

# CRITICAL: Set environment variables FIRST before any instrumentation
print("🔧 Setting up OpenTelemetry environment variables...")

# Enable content capture (prompts and responses) - MUST be set before instrumentation
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"  # Keep for backward compatibility

# Let the Azure SDK know we want to use OpenTelemetry
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"

# Additional OpenTelemetry configuration for better trace visibility
os.environ["OTEL_PYTHON_LOG_CORRELATION"] = "true"
os.environ["OTEL_PYTHON_LOG_LEVEL"] = "info"

print("✅ Environment variables configured for content capture and tracing")

# Microsoft's exact approach: Instrument OpenAI SDK first
try:
    from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
    OpenAIInstrumentor().instrument()
    print("✅ OpenAI v2 instrumentation enabled (Microsoft approach).")
except ImportError:
    print("⚠️ opentelemetry-instrumentation-openai-v2 not available")
    print("💡 Installing required package...")

# Then instrument Azure AI Inference
AIInferenceInstrumentor().instrument()
print("✅ Azure AI Inference instrumentation enabled.")
print("✅ Content recording enabled for traces.")

### 3.1.2 Point Traces to Console or Local OTLP
The simplest is to pipe them to **stdout**. If you want to send them to **Prompty** or **Aspire**, specify the local OTLP endpoint URL (usually `"http://localhost:4317"` or similar).

In [ ]:
# Console tracing setup - following Microsoft's official approach
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter

# Set up console tracing for local debugging
try:
    # Initialize tracer provider if not already done
    if not hasattr(trace, '_TRACER_PROVIDER') or trace._TRACER_PROVIDER is None:
        tracer_provider = TracerProvider()
        trace.set_tracer_provider(tracer_provider)
    
    # Add console exporter to see traces in stdout using Microsoft's recommended approach
    console_exporter = ConsoleSpanExporter()
    simple_processor = SimpleSpanProcessor(console_exporter)
    trace.get_tracer_provider().add_span_processor(simple_processor)
    
    print("✅ Console telemetry tracing enabled (Microsoft official approach).")
except Exception as e:
    print(f"⚠️ Could not set up console tracing: {e}")
    print("💡 Proceeding without console tracing - telemetry may still work via other means")

try:
    # Use the project client to get Azure OpenAI client - this is the recommended approach
    openai_client = project_client.get_openai_client(api_version="2024-10-21")
    user_prompt = "What's a simple 5-minute warmup routine?"
    local_resp = openai_client.chat.completions.create(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        messages=[{"role": "user", "content": user_prompt}]
    )
    print("\n🤖 Response:", local_resp.choices[0].message.content)
    print("🔍 Check console output above for trace information")
except Exception as exc:
    print(f"❌ Error in local-tracing example: {exc}")

## 4. Azure Monitor Tracing (Application Insights)
Now we'll set up tracing to **Application Insights**, which will forward your logs to the **Azure AI Foundry** **Tracing** page.

**Steps**:
1. In AI Foundry, go to your project’s **Tracing** tab, attach (or create) an **Application Insights** resource.
2. In code, call `project_client.telemetry.get_connection_string()` to retrieve the instrumentation key.
3. Use `azure.monitor.opentelemetry.configure_azure_monitor(...)` with that connection.
4. Make an inference call -> logs appear in the Foundry portal (and in Azure Monitor itself).


In [ ]:
# %pip install azure-monitor-opentelemetry

In [ ]:
# # Install additional Azure Monitor components for manual setup if needed
# %pip install azure-monitor-opentelemetry-exporter

In [ ]:
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace

# Get Application Insights connection string and configure Azure Monitor
print("🔧 Setting up Azure Monitor tracing for AI Foundry...")

try:
    # Get connection string from project client
    connection_string = project_client.telemetry.get_application_insights_connection_string()
    
    if connection_string:
        print("✅ Retrieved Application Insights connection string")
        
        # Configure Azure Monitor with the connection string
        configure_azure_monitor(
            connection_string=connection_string,
            enable_live_metrics=True
        )
        print("✅ Azure Monitor configured - traces will now be sent to AI Foundry!")
        
        # Now send a test trace to verify AI Foundry integration
        print("\n🧪 Sending test trace to AI Foundry...")
        
        tracer = trace.get_tracer(__name__)
        with tracer.start_as_current_span("ai_foundry_debug_trace") as span:
            # Use the exact attribute pattern that worked before
            span.set_attribute("ai.operation.name", "notebook_debug")
            span.set_attribute("service.name", "azure-ai-foundry-notebook")
            span.set_attribute("gen_ai.system", "openai")
            span.set_attribute("gen_ai.operation.name", "chat")
            span.set_attribute("operation.type", "debug_test")
            
            # Make an actual OpenAI call to generate a real trace
            client = project_client.get_openai_client(api_version="2024-10-21")
            response = client.chat.completions.create(
                model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
                messages=[
                    {"role": "user", "content": "This is a debug trace to verify Azure AI Foundry integration"}
                ]
            )
            
            # Add response details to span
            span.set_attribute("response.finish_reason", response.choices[0].finish_reason)
            span.set_attribute("response.model", response.model)
            
            print(f"✅ Debug trace sent with OpenAI call")
            print(f"🤖 Response: {response.choices[0].message.content[:100]}...")
        
        # Force flush to ensure immediate sending
        if hasattr(trace.get_tracer_provider(), 'force_flush'):
            trace.get_tracer_provider().force_flush(timeout_millis=5000)
            print("🚀 Traces flushed to Azure Monitor")
        
        print("\n🎯 Check Azure AI Foundry Tracing tab for:")
        print("   Operation Name: 'ai_foundry_debug_trace'")
        print("   Should appear within 5-10 minutes")
        print("\n💡 This uses the same pattern that worked before!")
        
    else:
        print("❌ No Application Insights connection string found")
        print("💡 Please ensure your AI Foundry project has Application Insights connected")
        
except Exception as e:
    print(f"❌ Failed to configure Azure Monitor: {e}")
    print("💡 Please check your AI Foundry project's Application Insights connection")

# 5. Agent-based Example
We'll now create a **Health Resource Agent** that references sample docs about recipes or guidelines, then demonstrate:
1. Creating an Agent with instructions.
2. Creating a conversation thread.
3. Running multi-step queries with **observability** enabled.
4. Optionally cleaning up resources at the end.

> The agent approach is helpful when you want more sophisticated conversation flows or **tool usage** (like file search).

## 5.1 Create Sample Files & Vector Store
We'll create dummy `.md` files about recipes/guidelines, then push them into a **vector store** so our agent can do semantic search.

(*This portion is a quick summary—see [the other file-search tutorial] if you need more details.)

In [ ]:
def create_sample_files():
    """Create some local .md files with sample text."""
    recipes_md = (
        """# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"""
    )

    guidelines_md = (
        """# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"""
    )

    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]

sample_files = create_sample_files()

def create_vector_store(files, store_name="my_health_resources"):
    try:
        uploaded_ids = []
        for fp in files:
            # Use the correct API pattern from the working notebooks
            upl = project_client.agents.files.upload_and_poll(
                file_path=fp,
                purpose="assistants"  # Use string instead of FilePurpose enum
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Create vector store from these file IDs using correct API pattern
        vs = project_client.agents.vector_stores.create_and_poll(
            file_ids=uploaded_ids,
            name=store_name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []

vector_store, file_ids = None, []
if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, store_name="health_resources_example")

## 5.2 Create a Health Resource Agent
We'll create a **FileSearchTool** referencing the vector store, then create an agent with instructions that it should:
1. Provide disclaimers.
2. Offer general nutrition or recipe tips.
3. Cite sources if possible.
4. Encourage professional consultation for deeper medical advice.


In [ ]:
def create_health_agent(vs_id):
    try:
        # Create agent using the working pattern from file-search notebook
        # Use tools as simple dictionary instead of FileSearchTool class
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="health-search-agent",
            instructions="""
                You are a health resource advisor with access to dietary and recipe files.
                You:
                1. Always present disclaimers (you're not a medical professional)
                2. Provide references to files when possible
                3. Focus on general nutrition or recipe tips.
                4. Encourage professional consultation for more detailed advice.
            """,
            tools=[{"type": "file_search"}]  # Use dictionary instead of FileSearchTool class
        )
        print(f"🎉 Created agent '{agent.name}' with ID: {agent.id}")
        print("📋 Vector store will be attached at message level for better compatibility")
        return agent
    except Exception as e:
        print(f"❌ Error creating health agent: {e}")
        return None

health_agent = None
if vector_store:
    health_agent = create_health_agent(vector_store.id)

## 5.3 Using the Agent with Enhanced Tracing
Let's create a new conversation **thread** and ask the agent some questions. We've enhanced this section to add **explicit tracing spans** around agent operations to ensure they appear in **Azure AI Foundry**.

**Enhanced Tracing Features**:
- Custom spans for agent demo, queries, and run execution
- Detailed attributes including agent ID, thread ID, query text, and file attachment counts
- Error tracking and status monitoring
- Hierarchical span structure for better trace visualization

In [ ]:
def create_thread():
    try:
        # Use the correct API pattern for thread creation
        thread = project_client.agents.threads.create()
        print(f"📝 Created new thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Could not create thread: {e}")
        return None

def ask_question_with_files(thread_id, agent_id, user_question, file_ids_list):
    try:
        # Create explicit tracing span for agent operations
        from opentelemetry import trace
        tracer = trace.get_tracer(__name__)
        
        with tracer.start_as_current_span("agent_file_search_query") as span:
            # Add custom attributes for better tracing visibility
            span.set_attribute("operation.type", "agent_file_search")
            span.set_attribute("agent.id", agent_id)
            span.set_attribute("thread.id", thread_id)
            span.set_attribute("query.text", user_question)
            span.set_attribute("file_attachments.count", len(file_ids_list))
            
            # Create message with file attachments for search - using working pattern
            attachments = []
            for file_id in file_ids_list:
                attachments.append({
                    "file_id": file_id,
                    "tools": [{"type": "file_search"}]
                })
            
            # Add user message with file attachments
            msg = project_client.agents.messages.create(
                thread_id=thread_id,
                role="user",
                content=user_question,
                attachments=attachments
            )
            print(f"User asked: '{user_question}' with {len(file_ids_list)} file attachments")
            
            # Create & process a run with tracing
            with tracer.start_as_current_span("agent_run_execution") as run_span:
                run_span.set_attribute("agent.model", os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"))
                
                run = project_client.agents.runs.create_and_process(
                    thread_id=thread_id,
                    agent_id=agent_id
                )
                
                # Add run details to span
                run_span.set_attribute("run.id", run.id)
                run_span.set_attribute("run.status", run.status)
                
                print(f"Run finished with status: {run.status}")
                if run.last_error:
                    print("Error details:", run.last_error)
                    run_span.set_attribute("run.error", str(run.last_error))
                
                return run
                
    except Exception as e:
        print(f"❌ Error asking question: {e}")
        if 'span' in locals():
            span.set_attribute("error", True)
            span.set_attribute("error.message", str(e))
        return None

if health_agent:
    # Create thread with tracing context
    from opentelemetry import trace
    tracer = trace.get_tracer(__name__)
    
    with tracer.start_as_current_span("health_agent_demo") as demo_span:
        demo_span.set_attribute("operation.type", "agent_demo")
        demo_span.set_attribute("agent.name", "health-search-agent")
        
        thread = create_thread()
        if thread:
            # Let's ask a few sample questions with file search - all within tracing context
            queries = [
                "Could you suggest a gluten-free lunch recipe?",
                "Show me some heart-healthy meal ideas.",
                "What guidelines do you have for someone with diabetes?"
            ]
            
            demo_span.set_attribute("queries.count", len(queries))
            
            for idx, q in enumerate(queries):
                if file_ids:
                    with tracer.start_as_current_span(f"query_{idx+1}") as query_span:
                        query_span.set_attribute("query.index", idx + 1)
                        query_span.set_attribute("query.text", q)
                        ask_question_with_files(thread.id, health_agent.id, q, file_ids)

    print("\n🎯 Agent traces should now be visible in Azure AI Foundry!")
    print("   Look for traces with operation names:")
    print("   - 'health_agent_demo' (overall demo)")
    print("   - 'agent_file_search_query' (each question)")
    print("   - 'agent_run_execution' (agent processing)")

### 5.3.1 Viewing the Conversation & Tracing Status
We can retrieve the conversation messages to see how the agent responded, check if it cited file passages, and **verify that our tracing is working properly**.

**🔍 Tracing Troubleshooting**:

If traces don't appear in Azure AI Foundry, check:

1. **Application Insights Connection**: Ensure your AI Foundry project has an Application Insights resource connected
2. **Connection String**: The `project_client.telemetry.get_connection_string()` should return a valid connection string
3. **Time Delay**: Traces can take 2-5 minutes to appear in the portal
4. **Trace Names**: Look for these specific operation names:
   - `health_agent_demo` (overall agent demo)
   - `agent_file_search_query` (each question asked)
   - `agent_run_execution` (agent processing)
   - `display_conversation` (conversation display)

5. **Manual Verification**: Check Application Insights directly in Azure Portal if AI Foundry doesn't show traces
6. **Environment Variables**: Ensure `OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT=true` is set

In [ ]:
def display_thread(thread_id):
    try:
        # Add tracing context for message retrieval
        from opentelemetry import trace
        tracer = trace.get_tracer(__name__)
        
        with tracer.start_as_current_span("display_conversation") as span:
            span.set_attribute("operation.type", "display_thread")
            span.set_attribute("thread.id", thread_id)
            
            # Use the correct API pattern for listing messages
            messages = project_client.agents.messages.list(thread_id=thread_id)
            print("\n🗣️ Conversation:")
            
            # Convert to list and reverse to show chronological order
            message_list = list(messages)
            span.set_attribute("messages.count", len(message_list))
            
            for m in reversed(message_list):
                if m.content:
                    last_content = m.content[-1]
                    if hasattr(last_content, "text"):
                        print(f"[{m.role.upper()}]: {last_content.text.value}\n")

            print("\n📎 Checking for citations...")
            citation_count = 0
            for m in message_list:
                if m.content:
                    for content_item in m.content:
                        if hasattr(content_item, "text") and hasattr(content_item.text, "annotations"):
                            for annotation in content_item.text.annotations:
                                citation_count += 1
                                if hasattr(annotation, "file_citation"):
                                    print(f"- Citation {citation_count}: '{annotation.text}' from file ID: {annotation.file_citation.file_id}")
                                else:
                                    print(f"- Citation {citation_count}: '{annotation.text}'")
            
            span.set_attribute("citations.count", citation_count)
            
            if citation_count == 0:
                print("No explicit citations found - checking for file content usage...")
                content_match_count = 0
                for m in message_list:
                    if m.role == "assistant" and m.content:
                        for content_item in m.content:
                            if hasattr(content_item, "text"):
                                text = content_item.text.value.lower()
                                if any(keyword in text for keyword in ['quinoa', 'salmon', 'gluten-free', 'diabetic', 'heart-healthy']):
                                    print(f"✅ Agent appears to be using file content (found relevant keywords)")
                                    content_match_count += 1
                                    break
                
                span.set_attribute("content_matches.count", content_match_count)
                                
    except Exception as e:
        print(f"❌ Could not display thread: {e}")
        import traceback
        traceback.print_exc()
        
        # Add error to span if it exists
        if 'span' in locals():
            span.set_attribute("error", True)
            span.set_attribute("error.message", str(e))

# If we created a thread above, let's read it with tracing
if 'health_agent' in globals() and health_agent and 'thread' in globals() and thread:
    print("\n" + "="*50)
    print("📊 DISPLAYING CONVERSATION WITH TRACING")
    print("="*50)
    display_thread(thread.id)
    
    print("\n🔍 **TRACING STATUS CHECK**:")
    print(f"✅ Azure Monitor connection: {'✓' if 'connection_string' in globals() and connection_string else '✗'}")
    print(f"✅ OpenTelemetry tracer: {'✓' if 'tracer' in globals() else '✗'}")
    print(f"✅ Agent operations traced: ✓ (with custom spans)")
    print(f"✅ Thread ID for reference: {thread.id}")
    
    print("\n🎯 **WHERE TO FIND TRACES**:")
    print("1. Go to https://ai.azure.com")  
    print("2. Navigate to your project")
    print("3. Click 'Tracing' in the left sidebar")
    print("4. Look for these trace operation names:")
    print("   - 'health_agent_demo' (overall demo)")
    print("   - 'agent_file_search_query' (individual questions)")
    print("   - 'agent_run_execution' (agent processing)")
    print("   - 'display_conversation' (this display function)")
    print("5. Filter by time range if needed (last 15-30 minutes)")

# 6. Cleanup
If desired, we can remove the vector store, files, and agent to keep things tidy. (In a real solution, you might keep them around.)

In [ ]:
def cleanup_resources():
    try:
        # Use the correct API pattern for cleanup
        if 'vector_store' in globals() and vector_store:
            project_client.agents.vector_stores.delete(vector_store.id)
            print("🗑️ Deleted vector store.")

        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.files.delete(fid)
            print("🗑️ Deleted uploaded files.")

        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health agent.")

        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")
    except Exception as e:
        print(f"❌ Error cleaning up: {e}")


cleanup_resources()

# 🎉 Wrap-Up
We've demonstrated:
1. **Basic LLM calls** with `AIProjectClient`.
2. **Listing connections** in your Azure AI Foundry project.
3. **Observability & tracing** in both local (console, OTLP endpoint) and cloud (App Insights) contexts.
4. A quick **Agent** scenario that uses a vector store for searching sample docs.

## Next Steps
- Check the **Tracing** tab in your Azure AI Foundry portal to see the logs.
- Explore advanced queries in Application Insights.
- Use [Prompty](https://github.com/microsoft/prompty) or [Aspire](https://learn.microsoft.com/dotnet/aspire/) for local telemetry dashboards.
- Incorporate this approach into your **production** GenAI pipelines!

> 🏋️ **Health Reminder**: The LLM's suggestions are for demonstration only. For real health decisions, consult a professional.

Happy Observing & Tracing! 🎉